# APLICAÇÃO DO ALGORTIMO DE ÁRVORE DE DECISÃO

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler

Leitura do dataset final.

In [2]:
dataset = pd.read_csv('DATASET_FINAL_PARA_APLICAÇÃO_DOS_ALGORITIMOS.csv')
dataset.head()

,target,High,Low,Open,Close,Volume,Adj Close,Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa AC,...,Empréstimos e Financiamentos PNC,Outros PNC,Tributos Diferidos PNC,Provisões PNC,Participação dos Acionistas Não Controladores,Patrimônio Líquido,Capital Social Realizado,Reservas de Capital,Reservas de Lucros,Lucros/Prejuízos Acumulados
0,Bad,42.900002,19.440001,19.985001,22.750000,3.449634e+07,12.533106,2885705.0,1715015.0,374622.0,...,471957.0,17108.0,81381.776590,119916.0,1.0,1033561.0,415955.0,147287.0,2.581270e+05,234891.000000
1,Bad,15.240000,9.320000,13.980000,11.150000,4.231002e+07,11.624119,63253422.0,20793485.0,3111608.0,...,18649526.0,906897.0,944546.000000,2946308.0,19644613.0,11596044.0,6881998.0,-69861.0,4.148446e+06,198944.931086
2,Bad,27.309999,21.730000,24.750000,24.600000,1.827805e+07,23.324638,4910968.0,882855.0,238905.0,...,1739126.0,4326.0,70845.000000,14419.0,6617.0,2487188.0,1231313.0,473382.0,7.824930e+05,109693.674100
3,Bad,30.350000,26.270000,29.610001,27.139999,3.827353e+07,25.771338,3454612.0,1126358.0,191491.0,...,564759.0,67259.0,371338.000000,28151.0,1.0,1600873.0,1413608.0,20077.0,7.068100e+04,96429.000000
4,Bad,22.500000,11.583700,19.866100,14.940000,1.157091e+07,16.613333,8265026.0,3633042.0,123715.0,...,2646948.0,105024.0,16875.367971,320365.0,108.0,2861085.0,3632101.0,24773.0,5.024450e+05,-795460.000000


Só mostrando que o alvo a ser predito está balanceado.

In [3]:
dataset['target'].value_counts()

Good    657
Bad     657
Name: target, dtype: int64

Aqui normalizamos o dataset, trazendo para uma escala de 0 a 1, ainda que isso não faça muita diferença em árvore de decisão.

In [4]:
normalizador = MinMaxScaler(feature_range=(0,1))

normalizador_features = normalizador.fit_transform(dataset.drop('target',axis=1))

dataset_normalizador = pd.DataFrame(normalizador_features,columns=dataset.columns[1:])

X_train, X_test, y_train, y_test = train_test_split(normalizador_features, dataset['target'], test_size=0.2)

Aqui calculamos o tempos do processo de Grid Seach onde ele busca os melhores parâmetros que especulamos para cada hiperparâmetro.

Assim, printamos os melhores valores estimados.

In [5]:
%%time

decision_tree = DecisionTreeClassifier()

parameter = {'min_samples_split':[2,3,5,10,15],
             'max_depth':range(1,25,2),
             'min_samples_leaf':[1,2,5] }

dtree_gridcv = GridSearchCV(decision_tree, parameter, cv=10, scoring='accuracy')
dtree_gridcv.fit(X_train, y_train)
dtree_gridcv.best_estimator_



CPU times: user 34.7 s, sys: 7.47 ms, total: 34.7 s
Wall time: 34.7 s


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=15,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

Aqui, extraimos as predições feitas nos dados de teste.

In [6]:
predictions = dtree_gridcv.best_estimator_.predict(X_test)

## CLASSIFICAÇÃO. 

Classification report nos traz as principais métricas de desempenho do algoritmo nesses dados.

In [7]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

         Bad       0.73      0.79      0.76       127
        Good       0.79      0.73      0.76       136

    accuracy                           0.76       263
   macro avg       0.76      0.76      0.76       263
weighted avg       0.76      0.76      0.76       263



E ainda temos o desempenho com base no score do grid search com o cross validation que foi "acurácia".

In [8]:
dtree_gridcv.best_score_

0.7668886774500476

## Importar modelo para usarmos depois no processo de inferência.

In [9]:
import pickle 

In [10]:
s = pickle.dumps(dtree_gridcv)

In [11]:
from joblib import dump, load

In [12]:
dump(dtree_gridcv, 'ArvoreDeDecisão.joblib')

['ArvoreDeDecisão.joblib']